In [ ]:
!pip -q install opensearch-py requests -U

# 한글 데이터 인덱싱 비교

- 한글 데이터를 한글 토크나이저 유무에 따라 비교

## 1. 로컬에 Opensearch 띄우기

프로젝트 루트 폴더에서 아래 명령을 통해 도커로 Opensearch 를 실행한다.

```bash
$ docker compose up -d
```

## 2. 형태소 분석

- `_analyze` 엔드포인트를 통해 분석결과를 미리 확인해볼 수 있다.

In [82]:
import requests

In [83]:
host = 'localhost'
port = 9200

In [84]:
url = f"http://{host}:{port}/_analyze"
url

'http://localhost:9200/_analyze'

In [85]:
text = "<b>이천이십삼년 韓國</b>, 뿌리가 깊은 나무는 바람에 흔들리지 않는다"

- 내장 형태소 분석기

In [86]:
requests.get(url, json={"analyzer": "standard", "text": text}).json()

{'tokens': [{'token': 'b',
   'start_offset': 1,
   'end_offset': 2,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': '이천이십삼년',
   'start_offset': 3,
   'end_offset': 9,
   'type': '<HANGUL>',
   'position': 1},
  {'token': '韓',
   'start_offset': 10,
   'end_offset': 11,
   'type': '<IDEOGRAPHIC>',
   'position': 2},
  {'token': '國',
   'start_offset': 11,
   'end_offset': 12,
   'type': '<IDEOGRAPHIC>',
   'position': 3},
  {'token': 'b',
   'start_offset': 14,
   'end_offset': 15,
   'type': '<ALPHANUM>',
   'position': 4},
  {'token': '뿌리가',
   'start_offset': 18,
   'end_offset': 21,
   'type': '<HANGUL>',
   'position': 5},
  {'token': '깊은',
   'start_offset': 22,
   'end_offset': 24,
   'type': '<HANGUL>',
   'position': 6},
  {'token': '나무는',
   'start_offset': 25,
   'end_offset': 28,
   'type': '<HANGUL>',
   'position': 7},
  {'token': '바람에',
   'start_offset': 29,
   'end_offset': 32,
   'type': '<HANGUL>',
   'position': 8},
  {'token': '흔들리지',
   'start_offset': 33

- 노리(Nori) 한글 형태소 분석기

In [87]:
requests.get(url, json={"analyzer": "nori", "text": text}).json()

{'tokens': [{'token': 'b',
   'start_offset': 1,
   'end_offset': 2,
   'type': 'word',
   'position': 0},
  {'token': '천',
   'start_offset': 4,
   'end_offset': 5,
   'type': 'word',
   'position': 2},
  {'token': '이',
   'start_offset': 5,
   'end_offset': 6,
   'type': 'word',
   'position': 3},
  {'token': '십',
   'start_offset': 6,
   'end_offset': 7,
   'type': 'word',
   'position': 4},
  {'token': '삼',
   'start_offset': 7,
   'end_offset': 8,
   'type': 'word',
   'position': 5},
  {'token': '년',
   'start_offset': 8,
   'end_offset': 9,
   'type': 'word',
   'position': 6},
  {'token': '한국',
   'start_offset': 10,
   'end_offset': 12,
   'type': 'word',
   'position': 7},
  {'token': 'b',
   'start_offset': 14,
   'end_offset': 15,
   'type': 'word',
   'position': 8},
  {'token': '뿌리',
   'start_offset': 18,
   'end_offset': 20,
   'type': 'word',
   'position': 9},
  {'token': '깊',
   'start_offset': 22,
   'end_offset': 23,
   'type': 'word',
   'position': 11},
  {'token

## 3. 인덱싱

- 인덱스 만들기
- 실제 검색결과에 어떤 영향을 미치는지 보기

In [ ]:
from opensearchpy import OpenSearch, Search

In [89]:
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True,
    use_ssl = False,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [90]:
index_name = 'korean-tokenizer-test'
index_body = {
    'settings': {
        'index': {
            'number_of_shards': 1,
        },
        "analysis": {
          "analyzer": {
            "nori_analyzer": {
              "type": "custom",
              "char_filter": [
                  "html_strip",
              ],
              "tokenizer": "nori_tokenizer",
              "filter": [
                  "nori_number", "nori_readingform", "lowercase"
              ],
              "decompound_mode": "mixed"
            }
          }
        }
    },
    'mappings': {
        'properties': {
            'text_plain': {'type': 'text'},
            'text_nori': {'type': 'text', 'analyzer': 'nori_analyzer'},
        },
    },
}

response = client.indices.create(index_name, body=index_body)
response

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'korean-tokenizer-test'}

In [91]:
client.indices.get(index_name)

{'korean-tokenizer-test': {'aliases': {},
  'mappings': {'properties': {'text_nori': {'type': 'text',
     'analyzer': 'nori_analyzer'},
    'text_plain': {'type': 'text'}}},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '1',
    'provided_name': 'korean-tokenizer-test',
    'creation_date': '1732112990520',
    'analysis': {'analyzer': {'nori_analyzer': {'filter': ['nori_number',
        'nori_readingform',
        'lowercase'],
       'decompound_mode': 'mixed',
       'char_filter': ['html_strip'],
       'type': 'custom',
       'tokenizer': 'nori_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': 'WYs-_-_oS0W0BIztl204Fw',
    'version': {'created': '136397827'}}}}}

- 데이터 추가

In [92]:
client.index(index_name, body={'text_plain': text, 'text_nori': text})

{'_index': 'korean-tokenizer-test',
 '_id': 'GZH6SZMBIi5xLZLK2Hgr',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [93]:
# client.indices.delete(index=index_name)

**char_filter**
- html_strip: html 태그를 제거하고, &nbsp; 등을 문자로 디코딩

**filter**
- nori_readingform: 한자를 한글로 변환
- nori_number: 숫자 변환

> bold 태그가 사라지고 `2023년 한국` 으로 저장된 것을 볼 수 있다.

In [95]:
client.indices.analyze(
    index=index_name,
    body={"text": text, "analyzer": "nori_analyzer"}
)

{'tokens': [{'token': '2023',
   'start_offset': 3,
   'end_offset': 8,
   'type': 'word',
   'position': 0},
  {'token': '년',
   'start_offset': 8,
   'end_offset': 9,
   'type': 'word',
   'position': 1},
  {'token': '한국',
   'start_offset': 10,
   'end_offset': 16,
   'type': 'word',
   'position': 2},
  {'token': '뿌리',
   'start_offset': 18,
   'end_offset': 20,
   'type': 'word',
   'position': 3},
  {'token': '가',
   'start_offset': 20,
   'end_offset': 21,
   'type': 'word',
   'position': 4},
  {'token': '깊',
   'start_offset': 22,
   'end_offset': 23,
   'type': 'word',
   'position': 5},
  {'token': '은',
   'start_offset': 23,
   'end_offset': 24,
   'type': 'word',
   'position': 6},
  {'token': '나무',
   'start_offset': 25,
   'end_offset': 27,
   'type': 'word',
   'position': 7},
  {'token': '는',
   'start_offset': 27,
   'end_offset': 28,
   'type': 'word',
   'position': 8},
  {'token': '바람',
   'start_offset': 29,
   'end_offset': 31,
   'type': 'word',
   'position': 9

## 3. 검색하기

- plain 은 문서에서 `흔들리면` 으로 인덱싱 되었기 때문에 부분 단어인 `흔들리` 는 검색 안됨
- nori 문서는 `흔들리` + `지` 으로 인덱싱 되었기 때문에, `흔들리` 토큰이 일치하므로 검색 가능

In [96]:
s = Search(using=client, index=index_name).query("match", text_plain="흔들리")
result = s.execute()
result.hits.hits

[]

In [97]:
s = Search(using=client, index=index_name).query("match", text_nori="흔들리면")
result = s.execute()
result.hits.hits

[{'_index': 'korean-tokenizer-test', '_id': 'GZH6SZMBIi5xLZLK2Hgr', '_score': 0.2876821, '_source': {'text_plain': '<b>이천이십삼년 韓國</b>, 뿌리가 깊은 나무는 바람에 흔들리지 않는다', 'text_nori': '<b>이천이십삼년 韓國</b>, 뿌리가 깊은 나무는 바람에 흔들리지 않는다'}}]

- 숫자로 써도 nori_number 필터에 의해 인덱싱이 숫자로 되어 있는 것을 확인 

In [101]:
s = Search(using=client, index=index_name).query("match", text_plain="2023")
result = s.execute()
result.hits.hits

[]

In [98]:
s = Search(using=client, index=index_name).query("match", text_nori="2023")
result = s.execute()
result.hits.hits

[{'_index': 'korean-tokenizer-test', '_id': 'GZH6SZMBIi5xLZLK2Hgr', '_score': 0.2876821, '_source': {'text_plain': '<b>이천이십삼년 韓國</b>, 뿌리가 깊은 나무는 바람에 흔들리지 않는다', 'text_nori': '<b>이천이십삼년 韓國</b>, 뿌리가 깊은 나무는 바람에 흔들리지 않는다'}}]

- plain 은 `<b>` 태그가 `b` 로 인덱싱 되므로 검색 가능 
- nori 는 태그는 삭제되어 인덱싱 되도록 설정했기 때문에 검색 불가능

In [102]:
s = Search(using=client, index=index_name).query("match", text_plain="<b>")
result = s.execute()
result.hits.hits

[{'_index': 'korean-tokenizer-test', '_id': 'GZH6SZMBIi5xLZLK2Hgr', '_score': 0.39556286, '_source': {'text_plain': '<b>이천이십삼년 韓國</b>, 뿌리가 깊은 나무는 바람에 흔들리지 않는다', 'text_nori': '<b>이천이십삼년 韓國</b>, 뿌리가 깊은 나무는 바람에 흔들리지 않는다'}}]

In [103]:
s = Search(using=client, index=index_name).query("match", text_nori="<b>")
result = s.execute()
result.hits.hits

[]